In [1]:
import os
from pydub import AudioSegment
from pydub.silence import split_on_silence
import speech_recognition as sr

In [2]:
def fragmentar_audio(audio_selected):
    sound_file = AudioSegment.from_wav(audio_selected + '.wav')
    # Si el texto existe se eliminara
    if os.path.exists(audio_selected + '.txt'):
        os.remove(audio_selected + '.txt')

    dBFS = sound_file.dBFS

    # Divide el audio en silencios de 300ms y deja un silencio al final del audio
    audio_chunks = split_on_silence(sound_file, min_silence_len=300, silence_thresh=dBFS - 16, keep_silence=1000)

    # Se crean los chunks de audio
    for i, chunk in enumerate(audio_chunks):

        out_file = "chunk{0}.wav".format(i)
        print("exporting", out_file)
        chunk.export(out_file, format="wav")

        r = sr.Recognizer()
        # r.energy_threshold = 400
        audio = sr.AudioFile('chunk' + str(i) + '.wav')

        with audio as source:
            audio_file = r.record(source)

            try:
                # Reconoce el texto del auido
                # text = r.recognize_google(audio_file, language='en')
                text = r.recognize_google(audio_file, language='es')
                print(text)

                # Crea un arvhivo de texto con el nombre del video
                with open(audio_selected + '.txt', "a") as receta_txt:
                    # Aniade el texto al final del archivo
                    receta_txt.write(text + " ")

            except Exception as e:
                print("Error")
                print(e)

        # Se elimina el chunk una vez se analiza
        if os.path.exists('chunk' + str(i) + ".wav"):
            os.remove('chunk' + str(i) + ".wav")
        else:
            print("El archivo que busca no existe")